<a href="https://colab.research.google.com/github/dominiquebuford/DL-individual/blob/main/DL_individual_converttococo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud init
from google.colab import files

# Upload the JSON key file
uploaded = files.upload()
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/service-account-key.json'

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%

Saving service-account-key.json to service-account-key.json


In [ ]:
!gsutil cp gs://dl-individual-project/annotations/filtered_annotations.csv /content/annotations.csv

Copying gs://dl-individual-project/annotations/filtered_annotations.csv...
/ [1 files][187.5 KiB/187.5 KiB]                                                
Operation completed over 1 objects/187.5 KiB.                                    


In [113]:
def createDictbackbone():
  keypoints = ['r ankle', 'r knee', 'r hip', 'l hip', 'l knee', 'l ankle', 'pelvis', 'thorax', 'upper neck', 'head top', 'r wrist', 'r elbow', 'r shoulder', 'l shoulder', 'l elbow', 'l wrist']
  skeleton = [
    # Lower body
    [13, 2],  # l shoulder -> r hip
    [2, 3],   # r hip -> l hip
    [3, 4],   # l hip -> l knee
    [4, 5],   # l knee -> l ankle
    [13, 1],  # l shoulder -> r knee
    [1, 0],   # r knee -> r ankle
    # Upper body
    [13, 8],  # l shoulder -> upper neck
    [8, 9],   # upper neck -> head top
    [13, 12], # l shoulder -> r shoulder
    [12, 11], # r shoulder -> r elbow
    [11, 10], # r elbow -> r wrist
    [13, 7],  # l shoulder -> thorax
    [7, 6],   # thorax -> pelvis
  ]
  cocoDict = {}
  cocoDict["categories"] = [{
    "supercategory": "person",
    "id": 1,
    "name": "person",
    "keypoints": keypoints,
    "skeleton": skeleton
  }]

  cocoDict['images'] = []
  cocoDict['annotations'] = []
  return cocoDict

In [ ]:
from google.cloud import storage
import pandas as pd

bucket_name = 'dl-individual-project'
images_folder = 'images/'
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blobs = bucket.list_blobs(prefix = images_folder)
blob_names = [blob.name[len(images_folder):] for blob in blobs]


846


In [111]:
def create_coco(imageNames, cocoDictionary):
  for i, name in enumerate(imageNames):
    fullName = f"images/{name}"
    tempName = f"./{name}"
    blob = bucket.blob(fullName)
    blob.download_to_filename(tempName)
    img = cv2.imread(tempName)
    # Get the height and width of the image
    height, width = img.shape[:2]

    #create input for the cocoDict['images'] list
    imageInput = {'file_name': name,
                'height': height,
                'width': width,
                'id': i}

    cocoDictionary['images'].append(imageInput)

    #create annotation entry for the associated row
    annotationsEntry = { 'iscrowd' : 0,
        "image_id" : i,
        "category_id" : 1,
        "id" : i
      }

    annotationsEntry['keypoints'] = []
    #create keypoints list of lists
    keyPointsList = df.loc[df['NAME']==name].loc[:, columnsKeylistNames].squeeze().tolist()
    for k in range(0,len(keyPointsList), 2):
      if keyPointsList[k] == -1:
       annotation = [keyPointsList[k], keyPointsList[k+1], 0]
      else:
        annotation = [keyPointsList[k], keyPointsList[k+1], 1]
      annotationsEntry['keypoints'].append(annotation)




    cocoDictionary['annotations'].append(annotationsEntry)
  return cocoDictionary





In [114]:
from sklearn.model_selection import train_test_split
import cv2
df = pd.read_csv("annotations.csv")
columnsKeylistNames = ['r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y', 'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X', 'l wrist_Y']
train_annotations, val_annotations = train_test_split(df['NAME'], test_size=0.2, random_state=42)
cocoDict_train = createDictbackbone()
cocoDict_train = create_coco(train_annotations, cocoDict_train)
cocoDict_val = createDictbackbone()

cocoDict_val = create_coco(val_annotations, cocoDict_val)


first 0
second 674


In [115]:
len(cocoDict_train['annotations'])

674

In [116]:
import json
with open("train_annotations.json", "w") as json_file:
        json.dump(cocoDict_train, json_file)

with open("val_annotations.json", "w") as json_file:
        json.dump(cocoDict_val, json_file)



In [117]:
blob = bucket.blob("annotations/train_annotations.json")

blob.upload_from_filename("train_annotations.json")

blob = bucket.blob("annotations/val_annotations.json")

blob.upload_from_filename("val_annotations.json")